#  Extract OFAM3 data

Get the data and compute the monthly climatology for various GWLs

Works with the following

Ocean Acidification daily fields pH and aragonite saturation state

SST daily fields

Temperature monthly fields where all depths, select depths and bottom depth can be output.
for this case the code writes to the same file


In [1]:
import xarray as xr
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import numpy as np
import matplotlib.pyplot as plt
import datetime
import flox
import flox.xarray
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter


In [2]:
import dask
# Set configuration options
dask.config.set({
    'distributed.comm.timeouts.connect': '90s',  # Timeout for connecting to a worker
    'distributed.comm.timeouts.tcp': '90s',  # Timeout for TCP communications
})

client = Client() # Client(threads_per_worker=1) # why?
client


2024-06-30 12:44:26,375 - distributed.preloading - INFO - Creating preload: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py
2024-06-30 12:44:26,380 - distributed.utils - INFO - Reload module schedplugin from .py file
2024-06-30 12:44:26,387 - distributed.preloading - INFO - Import preload module: /g/data/hh5/public/apps/dask-optimiser/schedplugin.py


Modifying workers


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /node/gadi-mmem-bdw-0003.gadi.nci.org.au/40197/proxy/8787/status,
Dashboard: /node/gadi-mmem-bdw-0003.gadi.nci.org.au/40197/proxy/8787/status,Workers: 32
Total threads: 32,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35729,Workers: 32
Dashboard: /node/gadi-mmem-bdw-0003.gadi.nci.org.au/40197/proxy/8787/status,Total threads: 32
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:40625,Total threads: 1
Dashboard: /node/gadi-mmem-bdw-0003.gadi.nci.org.au/40197/proxy/42243/status,Memory: 0 B
Nanny: tcp://127.0.0.1:37261,


In [3]:
# Function to calculate monthly climatology and convert to single precision
def monthly_climatology(ds, time_dim):
    """
    Calculate the monthly climatology for a given dataset.

    This function takes an xarray dataset and computes the monthly climatology
    by averaging data for each month over all years. The function adds a 'month'
    dimension to the dataset based on the provided time dimension.

    Parameters:
        ds (xarray.Dataset): The input dataset containing the time dimension.
        time_dim (str): The name of the time dimension in the dataset.

    Returns
        xarray.Dataset: Dataset with the mean values for each month across all years. 
        The data type of returned values is float32.
    """
    ds['month'] = ds[time_dim].dt.month
    climatology_cohorts = flox.xarray.xarray_reduce(
        ds,
        'month',
        func='mean',
        method='cohorts',
    )
    return climatology_cohorts.astype(np.float32)

In [4]:
# Set GWL periods
GWL_periods = {
    'current': ('1995-01-01', '2014-12-31'),
    'GW1p2': ('2001-01-01', '2020-12-31'),
    'GW1p5': ('2015-01-01', '2034-12-31'),
    'GW2p0': ('2030-01-01', '2049-12-31'),
    'GW3p0': ('2053-01-01', '2072-12-31'),
    'GW4p0': ('2074-01-01', '2093-12-31')
}

In [5]:
# Process and save climatology data
def process_climatology(ds, time_dim, start, end, variable, period):
    data = getattr(ds, variable).sel(**{time_dim: slice(start, end)}).persist() 
    clim = monthly_climatology(data, time_dim).persist()

    file_path = f'/g/data/ia39/ncra/ocean/{variable}_climatology_{period}.nc'
    
    # Save as netCDF
    clim.to_netcdf(file_path, compute=True)

In [9]:
%%time 
# Process OA fields 
# Directory paths for OA fields created
dir1 = '/scratch/xv83/rxm599/historical/'
dir2 = '/scratch/xv83/rxm599/future/'
dir0 = '/scratch/xv83/rxm599/pi/'

# Load datasets with chunking
dbgc1 = xr.open_mfdataset(dir1 + "files*.nc", parallel=True, chunks={'TIME41': 10})
dbgc2 = xr.open_mfdataset(dir2 + "files*.nc", parallel=True, chunks={'TIME41': 10})
dbgc0 = xr.open_mfdataset(dir0 + "files*.nc", parallel=True, chunks={'TIME41': 10})
# Process current period
process_climatology(dbgc1, 'TIME41', *GWL_periods['current'], 'OAR', 'current')
process_climatology(dbgc1, 'TIME41', *GWL_periods['current'], 'PH', 'current')

# Process PI
process_climatology(dbgc0, 'TIME41', *GWL_periods['current'], 'OAR', 'PI')
process_climatology(dbgc0, 'TIME41', *GWL_periods['current'], 'PH', 'PI')

# Process future
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        continue
    process_climatology(dbgc2, 'TIME41', start, end, 'OAR', period)
    process_climatology(dbgc2, 'TIME41', start, end, 'PH', period)

In [13]:
%%time
# Process SST fields 
# Directory paths for SST
dir1_new = '/g/data/fp2/OFAM3/jra55_historical.1/surface/'
dir2_new = '/g/data/fp2/OFAM3/jra55_rcp8p5/surface/'

# Load datasets with chunking
dsst1 = xr.open_mfdataset(dir1_new + 'ocean_temp_sfc_*', parallel=True, chunks={'Time': 10})
dsst2 = xr.open_mfdataset(dir2_new + 'ocean_temp_sfc*.nc', parallel=True, chunks={'Time': 10})

# Process SST
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        process_climatology(dsst1, 'Time', start, end, 'temp', period)
    else:
        process_climatology(dsst2, 'Time', start, end, 'temp', period)

In [7]:
%%time
# Process NPP fields 
# Directory paths for NPP 
dir1_new = '/g/data/fp2/OFAM3/jra55_historical.1/bgc_monthly_ave/'
dir2_new = '/g/data/fp2/OFAM3/jra55_rcp8p5/bgc_monthly_ave/'

# Load datasets with chunking
dsst1 = xr.open_mfdataset(dir1_new + 'ocean_bgc_2d_mth*.nc', parallel=True, chunks={'Time': 10})
dsst2 = xr.open_mfdataset(dir2_new + 'ocean_bgc_2d_mth*.nc', parallel=True, chunks={'Time': 10})

# Process NPP
for period, (start, end) in GWL_periods.items():
    if period == 'current':
        process_climatology(dsst1, 'Time', start, end, 'pprod_gross_2d', period)
    else:
        process_climatology(dsst2, 'Time', start, end, 'pprod_gross_2d', period)


CPU times: user 2min 6s, sys: 19.3 s, total: 2min 25s
Wall time: 3min 43s


In [22]:
%%time
# Process TEMP fields 
# Directory paths for Temp
dir1_new = '/g/data/fp2/OFAM3/jra55_historical.1/monthly_ave/'
dir2_new = '/g/data/fp2/OFAM3/jra55_rcp8p5/monthly_ave/'

# Load datasets with chunking
#dsst1 = xr.open_mfdataset(dir1_new + 'ocean_temp_mth*.nc', parallel=True, chunks={'Time': 10})
#dsst2 = xr.open_mfdataset(dir2_new + 'ocean_temp_mth*.nc', parallel=True, chunks={'Time': 10})

# select depth range 
da1=dsst1.sel(st_ocean=slice(0, 1000))
da2=dsst2.sel(st_ocean=slice(0, 1000))
# select a depth
da1=dsst1.isel(st_ocean=(14))  # near 100m
da2=dsst2.isel(st_ocean=(14))
# fill in depth to get the bottom value
db1=dsst1.ffill(dim="st_ocean", limit=None).isel(st_ocean=-1)
db2=dsst2.ffill(dim="st_ocean", limit=None).isel(st_ocean=-1)

CPU times: user 77.4 ms, sys: 24.9 ms, total: 102 ms
Wall time: 100 ms


In [25]:
%%time
# Can works on all depths (dsst1 dsst2)
# Reduced depths (da1 da2)
# Bottom depth (db1 db2)
# Process TEMP 
for period, (start, end) in GWL_periods.items():
    if period == 'current':
#        process_climatology(dsst1, 'Time', start, end, 'temp', period)
#        process_climatology(da1, 'Time', start, end, 'temp', period)
        process_climatology(db1, 'Time', start, end, 'temp', period)
        print(period)
    else:
#        process_climatology(dsst2, 'Time', start, end, 'temp', period)
#        process_climatology(da2, 'Time', start, end, 'temp', period)
        process_climatology(db2, 'Time', start, end, 'temp', period)
        print(period)

# Need to move the files to correct name (bot_temp directory ) 

current
GW1p2
GW1p5
GW2p0
GW3p0
GW4p0
CPU times: user 2min 6s, sys: 29.8 s, total: 2min 35s
Wall time: 4min 21s
